In [18]:
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import trt_pose.coco
import math
import os
import numpy as np
import traitlets


Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

In [19]:
path_models = ""
path_models = "trt_pose_hand/"

with open(path_models+'preprocess/hand_pose.json', 'r') as f:
    hand_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(hand_pose)
import trt_pose.models

num_parts = len(hand_pose['keypoints'])
num_links = len(hand_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
import torch


WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()


if not os.path.exists(path_models+'model/hand_pose_resnet18_att_244_244_trt.pth'):
    MODEL_WEIGHTS = path_models+'model/hand_pose_resnet18_att_244_244.pth'
    model.load_state_dict(torch.load(MODEL_WEIGHTS))
    import torch2trt
    model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)
    OPTIMIZED_MODEL = path_models+'model/hand_pose_resnet18_att_244_244_trt.pth'
    torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)


OPTIMIZED_MODEL = path_models+'model/hand_pose_resnet18_att_244_244_trt.pth'
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

<All keys matched successfully>

In [20]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology,cmap_threshold=0.15, link_threshold=0.15)
draw_objects = DrawObjects(topology)

In [21]:

import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [22]:
from trt_pose_hand.preprocessdata import preprocessdata
preprocessdata = preprocessdata(topology, num_parts)
from trt_pose_hand.gesture_classifier import gesture_classifier
gesture_classifier = gesture_classifier()

Next, let's define a function that will preprocess the image, which is originally in BGR8 / HWC format.

In [23]:
def draw_joints(image, joints):
    count = 0
    for i in joints:
        if i==[0,0]:
            count+=1
    if count>= 3:
        return 
    for i in joints:
        cv2.circle(image, (i[0],i[1]), 2, (0,0,255), 1)
    cv2.circle(image, (joints[0][0],joints[0][1]), 2, (255,0,255), 1)
    for i in hand_pose['skeleton']:
        if joints[i[0]-1][0]==0 or joints[i[1]-1][0] == 0:
            break
        cv2.line(image, (joints[i[0]-1][0],joints[i[0]-1][1]), (joints[i[1]-1][0],joints[i[1]-1][1]), (0,255,0), 1)

In [24]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30, capture_device=0)
#camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!


In [25]:
# import ipywidgets
# from IPython.display import display


# image_w = ipywidgets.Image(format='jpeg', width=224, height=224)
# display(image_w)

In [26]:
# MY SECTION!
import time
from multiprocessing import Process, Queue

monitor_data_path = "./monitor/monitor.json"
monitor_data = json.load(open(monitor_data_path))

q_tc2c = Queue()
time_start = time.time()

def mark_thumb(image, joints):
    for i in range(len(joints)):
        if i >= 0 and i < 5:
            cv2.circle(image, (joints[i][0],joints[i][1]), 2, (255,0,255), 5)
    cv2.line(image, (joints[0][0],joints[0][1]), (joints[1][0],joints[1][1]), (255,0,0), 2)


def get_w_ref_value(joints):
    max_w_ref = monitor_data["thumb"]["max_w_ref"]
    min_w_ref = monitor_data["thumb"]["min_w_ref"]
    
    p1 = np.array([joints[0][0],joints[0][1]])
    p2 = np.array([joints[1][0],joints[1][1]])
    diff = p2 - p1
    a = -np.arctan2(diff[1],diff[0])

    if a > np.pi/2:
        a = (np.pi-a) 
    elif a < -np.pi/2:
        a = (-np.pi-a) 

    if a >= 0:
        rad2w_ref_plus  = max_w_ref*2/np.pi
        w_ref = rad2w_ref_plus*a
    else:
        rad2w_ref_minus = min_w_ref*2/np.pi
        w_ref = rad2w_ref_minus*a

    return w_ref


def thumb_controller(image, joints):
    mark_thumb(image, joints)
    w_ref = get_w_ref_value(joints)
    if time.time() - time_start > 0.5:
        #print(w_ref)
        q_tc2c.put(w_ref)
# MY SECTION!

In [27]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)
    joints = preprocessdata.joints_inference(image, counts, objects, peaks)
    draw_joints(image, joints)
    
    # My code
    if np.all(joints) != [0.0]:
        w_ref = thumb_controller(image, joints)
    # My code
    
    #draw_objects(image, counts, objects, peaks)# try this for multiple hand pose prediction 
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])

In [28]:
def thumb_controller_run():
    execute({'new': camera.value})
    camera.observe(execute, names='value')

In [29]:
# execute({'new': camera.value})

In [30]:
# camera.observe(execute, names='value')

In [31]:
#camera.unobserve_all()

In [32]:
import ipywidgets
from IPython.display import display


image_w = ipywidgets.Image(format='jpeg', width=400, height=400)
display(image_w)

Image(value=b'', format='jpeg', height='400', width='400')

In [33]:
#camera.running = False

In [34]:
# MY SECTION!
from threading import Thread
from monitor.client import client
import numpy as np
import json

if __name__ == "__main__":
    monitor_data_path = "./monitor/monitor.json"
    monitor_data = json.load(open(monitor_data_path))
    
    process_client       = Thread(target=client,        
                                args=(q_tc2c, None, monitor_data, ), 
                                kwargs={"show":True, "debug":True})
    
    process_client.start()
    thumb_controller_run()

# MY SECTION!

Client: Running
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run 

Client: Connected with server 192.168.1.107!
Client: Main loop start


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/src/app/monitor/client.py", line 71, in send
    data = input()
EOFError: EOF when reading a line


Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

...

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1

Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1.107 in order to connect!
Client: Run server 192.168.1